<h1>Webscraping dos Índices</h1>

In [1]:
# Imports da selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [2]:
# Bibliotecas de uso geral
import os
from datetime import datetime, timedelta
import time

In [3]:
# Logger
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# Bibliotecas para manipulação de dados
import pandas as pd

In [5]:
# Bibliotecas para Comunicação com AWS
import boto3
import botocore
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv

In [6]:
def inicializa_sessao_s3():
    """
    Inicializa a sessão do boto3 com as credenciais da AWS.
    """
    # Gerenciar filepaths para onde estão as credenciais
    logger.info("Conectando na AWS...")
    notebook_path = os.getcwd()
    project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

    # Carregar variáveis de ambiente do arquivo .env
    os.chdir(project_path)
    load_dotenv()
    os.chdir(notebook_path)

    # Obter as credenciais da AWS do arquivo .env
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    aws_session_token = os.getenv('AWS_SESSION_TOKEN')

    # Iniciar a sessão do boto3 com as credenciais obtidas
    session = boto3.Session(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token
    )

    s3_client_session = session.client('s3')
    return s3_client_session

In [7]:
def configura_chrome_driver(download_dir:str=os.getcwd()) -> tuple:
    """
    Configura o Chrome Driver para download automático e execução em background.
    """
    # Configurações do Chrome Driver para download automático
    logger.info("Configurando o Chrome Driver...")
    chrome_options = webdriver.ChromeOptions()
    chrome_prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }

    # Configurações do Chrome Driver para execução em background
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.add_experimental_option("prefs", chrome_prefs)

    chrome_service = ChromeService(ChromeDriverManager().install())

    return chrome_service, chrome_options

In [8]:
def download_b3_file(
        url: str, date_str: str, 
        chrome_service: ChromeService, 
        chrome_options: webdriver.ChromeOptions) -> str:
    """
    Baixa o arquivo de índices da B3 para D-1 e retorna o caminho do arquivo baixado.
    """
    logger.info("Baixando o arquivo de indices da B3 via Selenium...")
    # Cria o diretório para salvar os arquivos csv versionados
    csv_dir = f"data/{date_str}/csv"
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)

    # Configura o diretório de download no Chrome
    prefs = {"download.default_directory": os.path.abspath(csv_dir)}
    chrome_options.add_experimental_option("prefs", prefs)
    
    # Inicia o WebDriver e acessa a URL
    driver = webdriver.Chrome(service=chrome_service, 
                              options=chrome_options)
    driver.get(url)
    
    try:
        # Espera o botão de download estar presente e clica nele
        wait = WebDriverWait(driver, 10)
        download_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Download")))
        download_button.click()
        
        # Espera o download completar
        time.sleep(5)  # Ajuste esse tempo se necessário

        # Verifica o path do arquivo baixado (último arquivo modificado na pasta de downloads)
        files = os.listdir(csv_dir)
        if not files:
            raise ValueError("Nenhum arquivo foi baixado.")
        downloaded_file_path = max(
            [os.path.join(csv_dir, f) for f in files], 
            key=os.path.getctime)
        
    finally:
        # Encerra o WebDriver
        driver.quit()
    
    return downloaded_file_path

In [12]:
def save_versioned_parquet(
        b3_csv_filepath:pd.DataFrame, 
        date_str:str) -> str:
    """
    Salva o dataframe em um parquet versionado e retorna o seu path.
    """
    logger.info("Criando o parquet versionado...")
    # Lê como DataFrame
    b3_df = pd.read_csv(b3_csv_filepath, 
                        sep=';',
                        skiprows=1, 
                        encoding='latin1',
                        usecols=[0, 1, 2, 3, 4])
    
    # Renomeia as colunas
    b3_df.columns = ['CODIGO', 'ACAO', 'TIPO', 'QUANTIDADE_TEORICA', 'PARTICIPACAO']

    # Dropa as duas últimas linhas que são os totais
    b3_df = b3_df.drop(b3_df.tail(2).index)   
    
    # Cria o diretório para salvar os arquivos parquet versionados
    parquet_dir = f"data/{date_str}/"
    if not os.path.exists(parquet_dir):
        os.makedirs(parquet_dir)

    # Transforma date_str para salvar parquet
    date_str = date_str.replace('/', '')

    # Nome do arquivo parquet e path completo
    parquet_file_name = f'IBOV_{date_str}.parquet'
    parquet_file_path = os.path.join(f"{parquet_dir}/{parquet_file_name}")

    b3_df.to_parquet(parquet_file_path, index=False)

    return parquet_file_path

In [10]:
def upload_to_s3(
        file_name:str, 
        bucket:str,
        s3_client: botocore.client,
        object_name:str=None):
    """
    Faz upload de arquivo para bucket do S3.

    :param file_name: Nome do arquivo a ser carregado
    :param bucket: Nome do bucket
    :param object_name: Nome do objeto no S3. Se não for fornecido, será usado o file_name
    :return: True se o upload foi bem-sucedido, caso contrário False
    """
    logger.info(f"Carregando o arquivo {file_name} para o bucket {bucket}...")
    if object_name is None:
        object_name = file_name

    try:
        s3_client.upload_file(file_name, bucket, object_name)
        logger.info(f"Arquivo {file_name} carregado com sucesso para o bucket {bucket} como {object_name}.")
        return True
    except FileNotFoundError:
        logger.error(f"Arquivo {file_name} não encontrado.")
        return False
    except NoCredentialsError:
        logger.error("Credenciais não disponíveis.")
        return False

In [11]:
def run_pipeline():
    """
    Core pipeline function.
    """

    # URL do conteúdo a ser baixado
    b3_url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
    bucket = 'raw-fiap-b3-projeto-02'

    # Obtendo data de ontem D-1
    yesterday = datetime.now() - timedelta(days=1)
    date_str = yesterday.strftime('%Y/%m/%d')

    ### PIPELINE CENTRAL ###

    # Inicializa a sessão do S3
    s3_client_session = inicializa_sessao_s3()

    # Configura o Chrome Driver
    chrome_service, chrome_options = configura_chrome_driver()
    
    # Obtém o CSV da B3
    b3_csv_filepath = download_b3_file(
        url=b3_url,
        date_str=date_str,
        chrome_service=chrome_service,
        chrome_options=chrome_options
        )
    
    # Salva o DataFrame em parquet versionado
    parquet_path = save_versioned_parquet(b3_csv_filepath,date_str)

    # # Faz upload do arquivo parquet para o S3
    upload_to_s3(parquet_path, bucket, s3_client_session)

    logger.info("DONE!")

# Executa o pipeline
run_pipeline()

2024-07-06 19:12:35,435 - INFO - Conectando na AWS...
2024-07-06 19:12:36,962 - INFO - Configurando o Chrome Driver...
2024-07-06 19:12:36,963 - INFO - ====== WebDriver manager ======
2024-07-06 19:13:14,416 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-06 19:13:14,981 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-06 19:13:15,080 - INFO - Driver [C:\Users\gufer\.wdm\drivers\chromedriver\win64\126.0.6478.126\chromedriver-win32/chromedriver.exe] found in cache
2024-07-06 19:13:15,102 - INFO - Baixando o arquivo de indices da B3 via Selenium...
2024-07-06 19:13:42,018 - INFO - Criando o parquet versionado...
2024-07-06 19:13:43,717 - INFO - Carregando o arquivo data/2024/07/05/parquet/IBOV_20240705.parquet para o bucket raw-fiap-b3-projeto-02...
2024-07-06 19:13:45,178 - INFO - Arquivo data/2024/07/05/parquet/IBOV_20240705.parquet carregado com sucesso para o bucket raw-fiap-b3-projeto-02 como data/2024/07/05/parquet/IBOV_20240705.parquet.
2024-07